Построем распределение клиентов по кривым.
- Использовать DEF, CLO, DLQ кривые
- Интерполировать и экстраполировать кривые для каждого значения PA/PD
- PA – x_score

In [1]:
import pandas as pd
import numpy as np

In [2]:
my_def_rate = pd.read_csv('data/def_rate.csv', index_col='statement_num').reindex(range(1, 102)).ffill()
my_clo_rate = pd.read_csv('data/clo_rate.csv', index_col='statement_num').reindex(range(1, 102)).ffill()
my_dlq_ratio = pd.read_csv('data/dlq_ratio.csv', index_col='statement_num').reindex(range(1, 102)).ffill()

In [4]:
import matplotlib.pyplot as plt

In [15]:
my_def_rate.head(24)

,pd = 0.011040,pd = 0.032685,pd = 0.068209
statement_num,,,
1,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000
4,0.014471,0.027497,0.036798
5,0.001552,0.005700,0.008627
6,0.002093,0.006226,0.009947
7,0.002496,0.006655,0.011099
8,0.002778,0.006999,0.012092
9,0.002960,0.007270,0.012938


In [3]:
def get_curves(clo_rate, def_rate, dlq_ratio, pd, pa):
    # выписываю значения метрик по известным кривым
    pds = [0.011040, 0.032685, 0.068209]
    pas = [0.133270, 0.220838, 0.379452]
    # нахожу, где относительно известных располагаются новые кривые
    pd_ind = np.searchsorted(pds, pd)
    pa_ind = np.searchsorted(pas, pa)

    # здесь считаю коэффициенты для интерполяции/экстраполяции
    def_k_01 = (def_rate.to_numpy().T[1] - def_rate.to_numpy().T[0]) / (pds[1] - pds[0])
    def_k_12 = (def_rate.to_numpy().T[2] - def_rate.to_numpy().T[1]) / (pds[2] - pds[1])
    dlq_k_01 = (dlq_ratio.to_numpy().T[1] - dlq_ratio.to_numpy().T[0]) / (pds[1] - pds[0])
    dlq_k_12 = (dlq_ratio.to_numpy().T[2] - dlq_ratio.to_numpy().T[1]) / (pds[2] - pds[1])
    clo_k_01 = (clo_rate.to_numpy().T[1] - clo_rate.to_numpy().T[0]) / (pas[1] - pas[0])
    clo_k_12 = (clo_rate.to_numpy().T[2] - clo_rate.to_numpy().T[1]) / (pas[2] - pas[1])

    # отдельно по каждому скору считаем новые кривые
    if pd_ind == 3:
        def_rate = def_rate.to_numpy().T[2] + def_k_12 * (pd - pds[2])
        dlq_ratio = dlq_ratio.to_numpy().T[2] + dlq_k_12 * (pd - pds[2])
    elif pd_ind == 2:
        def_rate = def_rate.to_numpy().T[1] + def_k_12 * (pd - pds[1])
        dlq_ratio = dlq_ratio.to_numpy().T[1] + dlq_k_12 * (pd - pds[1])
    elif pd_ind == 1:
        def_rate = def_rate.to_numpy().T[0] + def_k_01 * (pd - pds[0])
        dlq_ratio = dlq_ratio.to_numpy().T[0] + dlq_k_01 * (pd - pds[0])
    elif pd_ind == 0:
        def_rate = def_rate.to_numpy().T[0] - def_k_01 * (pd - pds[0])
        dlq_ratio = dlq_ratio.to_numpy().T[0] - dlq_k_01 * (pd - pds[0])

    if pa_ind == 3:
        clo_rate = clo_rate.to_numpy().T[2] + clo_k_12 * (pa - pas[2])
    elif pa_ind == 2:
        clo_rate = clo_rate.to_numpy().T[1] + clo_k_12 * (pa - pas[1])
    elif pa_ind == 1:
        clo_rate = clo_rate.to_numpy().T[0] + clo_k_01 * (pa - pas[0])
    elif pa_ind == 0:
        clo_rate = clo_rate.to_numpy().T[0] - clo_k_01 * (pa - pas[0])

    #  ограничиваю кривые
    clo_rate[clo_rate>1] = 1
    clo_rate[clo_rate<0] = 0
    def_rate[def_rate>1] = 1
    def_rate[def_rate<0] = 0
    dlq_ratio[dlq_ratio>1] = 1
    dlq_ratio[dlq_ratio<0] = 0
    
    return clo_rate, def_rate, dlq_ratio

In [4]:
def get_probs(clo_rate, def_rate, dlq_ratio):
    CUR = np.zeros(101)
    DLQ = np.zeros(101)
    DEF = np.zeros(101)
    CLO = np.zeros(101)
    ACT = np.zeros(101)
    
    DEF[0] = 0
    CLO[0] = clo_rate[0]
    ACT[0] = 1 - CLO[0] - DEF[0]
    for i in range(1, 101):
        DEF[i] = DEF[i-1] + ACT[i-1]*def_rate[i]
        CLO[i] = CLO[i-1] + ACT[i-1]*clo_rate[i]
        ACT[i] = 1 - CLO[i] - DEF[i]

    return CLO, DEF

In [5]:
# clo_rate, def_rate, dlq_ratio = get_curves(my_clo_rate, my_def_rate, my_dlq_ratio, 0.05, 0.001)
# CLO, DEF = get_probs(clo_rate, def_rate, dlq_ratio)  # доли от всех клиентов

In [1]:
def distribution_calc(self, pd, pa):
    '''
    Расчет распределения по статусам
    :param pd: вероятность выпасть к дефолт к 12-му месяцу
    :param pa: вероятность полного досрочного погашения к 6-му месяцу
    :param term: срок кредита
    :return:
    act_dist : np.array(101) : доля клиентов в статусе ACT на каждый месяц
    dlq_dist : np.array(101) : доля клиентов в статусе CUR на каждый месяц
    cur_dist : np.array(101) : доля клиентов в статусе DLQ на каждый месяц
    def_dist : np.array(101) : доля клиентов в статусе DEF на каждый месяц
    '''

    clo_rate, def_rate, dlq_ratio = get_curves(my_clo_rate, my_def_rate, my_dlq_ratio, pd, pa)
    CLO, DEF = get_probs(clo_rate, def_rate, dlq_ratio)

    ACT = 1 - CLO - DEF
    DLQ = ACT * dlq_ratio
    CUR = ACT - DLQ
    return ACT, DLQ, CUR, DEF